In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from google.cloud.firestore import Client as FirestoreClient

from chat_history_service import ChatHistoryService
from llm_request_service import LLMRequestService

In [ ]:
def get_chat_history_from_firestore(session_id: str) -> list[dict]:
    """
    Fetch chat history from Firestore for a given session ID.
    
    Args:
        session_id (str): The session ID to fetch chat history for.
    
    Returns:
        list[dict]: List of chat messages in the session.
    """
    chat_history_service = ChatHistoryService()
    chat_history = chat_history_service.get_chat_history_by_id(session_id)
    chat_history = sorted(chat_history, key=lambda x: x.get("create_time", ""))
    return chat_history

In [4]:
def call_llm_service(conversation: str, prompt: str = "") -> str:
    """
    Call the LLM service to summarize the chat history.
    Args:
        conversation (str): The chat history to summarize.
        prompt (str): The prompt to send to the LLM service.
            If not provided, the default prompt will be fetched from Firestore.
    Returns:
        str: The summary of the chat history.
    """
    llm_request_service = LLMRequestService()
    chat_history_summary = llm_request_service.summarize_conversation(
        conversation, prompt
    )
    return chat_history_summary

## Manage Prompt

In [ ]:
# Create a prompt that eventually contains the variable {text}
# The chat history will be concatenated into this prompt via this variable
prompt = ""

### Update the prompt in Firestore Database (code uses the uploaded one)

In [ ]:
firestore_client = FirestoreClient()
collection = firestore_client.collection("llm-prompts")
document = collection.document("fulfillment-webhook")
document.update({"summarize-chat-conversation": prompt})

# Gather Conversation

### Get conversation from session_id in Development

In [9]:
session_id = "20e62f9c-be3e-4d6e-978b-7d67dd5f10e8" # Replace with your actual session ID
conversation = get_chat_history_from_firestore(session_id)
conversation

'Mobiel\nvideoland\nAanvragen'